# 1. Evaluation du corpus
## a. Métriques adapatées

Afin d'évaluer les résultats d'un modèle avec notre corpus, nous avons regardé quelles sont les métriques d'évaluations utilisées dans le corpus de référence.

La métrique d'évaluation utilisée dans le corpus SQuaD est principalement le __F1-Score__. Une autre métrique d'évaluation en tâche de Q/A est l'__Exact Match__, qui indique s'il y a une correspondance exact entre ce qui est prédit et la vraie réponse.

Des explications sur la manière d'importer la métrique d'évaluation officielle de SQuaD se trouve à la page suivante : https://github.com/huggingface/evaluate/tree/main/metrics/squad .

In [6]:
#!pip install evaluate
from evaluate import load

# on importe la métrique de squad
squad_metric = load("squad")

# Nous avons d'une part la liste des prédictions données par un modèle :
predictions = []
# D'autre part la liste des réponses de références, dans le même ordre :
references = []

# Les prédictions et les références doivent avoir les formes suivantes :
# predictions = [{'prediction_text': '1976', 'id': '56e10a3be3433e1400422b22'}]
# references = [{'answers': {'answer_start': [97], 'text': ['1976']}, 'id': '56e10a3be3433e1400422b22'}]

# On calcule le résultat de F1-Score entre nos prédictions et nos références :
results = squad_metric.compute(predictions=predictions, references=references)

# Affichage des résultats :
result

ZeroDivisionError: float division by zero

## b. Métriques proposées

#### ROUGE : 

Nous pourrions utiliser la métrique d'évaluation __ROUGE (Recall-Oriented Understudy for Gisting Evaluation)__. 
Cette métrique est généralement utilisée dans des tâches de résumé automatique et de traduction automatique. Cependant, cette métrique d'évaluation permet de comparer, dans le cas d'un résumé automatique, le taux de similarité entre le résumé de référence et le résumé produit. 
Nous pourrions ainsi comparer, dans le cas de notre tâche de Q/A, la réponse donnée par rapport à la réponse de référence.

#### WER : 
Une autre possibilité pourrait être de calculer le taux d'erreur mot. Il eput nous permettre de calculer le taux de mots non attendu dans la réponse, qui sont donc concidérés comme des erreurs.

Le calcul du Word Error Rate est le suivant : __WER = (S + D + I) / N__
- S : nombre de substitutions
- D : nombre de délétions
- I : nombre d'insertions

Exemple : 
- question : _"Que témoigne le sourire ?"_, vraie réponse : _"la sympathie"_, réponse produite : _"le plaisir"_
--> 2/2 de __S__, WER = 100%

- question : _"Que témoigne le sourire ?"_, vraie réponse : _"la sympathie"_, réponse produite : _"sympathie"_
--> 1/2 de __D__, WER = 50%

- question : _"Que témoigne le sourire ?"_, vraie réponse : _"la sympathie"_, réponse produite : _"témoignant en général de la sympathie"_
--> 4/2 de __I__, WER = 100%

- question : _"Que témoigne le sourire ?"_, vraie réponse : _"la sympathie"_, réponse produite : _"la sympathie"_
--> WER = 0%

In [37]:
# site inspiration code : https://thepythoncode.com/article/calculate-word-error-rate-in-python


def calcul_wer(reference: str, prediction: str):
    """
    Fonction qui calcule le taux d'erreur mot entre une référence et une prédiction.
    Parameters
    ----------
    reference : str
        la réponse de référence du corpus
    prediction : str
        la réponse produite par le modèle
    Returns
    -------
    wer : float
        le taux d'erreur mot entre la référence et la prédiction
    """
    # Récupération de liste de mot dans la référence et la prédiciton
    ref_words = reference.split()
    pred_words = prediction.split()

    # On compte le nombre de substitutions, délétions et insertions.
    substitutions = sum(1 for ref, pred in zip(ref_words, pred_words) if ref != pred)
    deletions = len(ref_words) - len(pred_words)
    insertions = len(pred_words) - len(ref_words)

    # Calcul du combre total de mots dans la référence
    total_words = len(ref_words)

    # Calcul du taux d'erreur mot :
    wer = (substitutions + deletions + insertions) / total_words

    return wer


reference_text = "la sympathie"
prediction_text = "sympathie"

wer_score = calcul_wer(reference_text, prediction_text)
print(
    "Le taux d'erreur mot (WER) entre '",
    reference_text,
    "' et '",
    prediction_text,
    "' est :",
    wer_score,
)

Le taux d'erreur mot (WER) entre ' la sympathie ' et ' sympathie ' est : 0.5


In [24]:
# imaginons que nous avons la liste des réponses produites par le modèle
# et la liste des vraies réponses de notre corpus
# les deux listes ci-dessous sont vides, comme nous ne lançons aucun modèle

# liste des vraies réponses, qu'on récupère de notre corpus :
reponses_references = []

# liste des réponses produites par le modèle :
reponses_predites = []

# on applique notre fonction de calcul du WER à l'ensemble des réponses :
for i in range(len(reponses_references)):
    calcul_wer(reponses_references[i], reponses_predites[i])

In [36]:
# Illustration de comment récupérer l'ensemble des réponses de notre corpus :

import pandas as pd
import re

corpus = pd.read_csv("../data/clean/clean_datas_chat.csv")

reponses = list(corpus.answer)

reponses_references = []

for reference in reponses:
    liste_reponse = re.findall(r"{'text': '(.+?)', 'answer_start':.+", reference)
    if len(liste_reponse) != 0:
        reponse = liste_reponse[0]
    else:
        reponse = ""
    reponses_references.append(reponse)

reponses_references

['forces de  tension superficielle',
 'Pseudaelurus',
 'la sympathie',
 '',
 'en 1850',
 'la  ceinture scapulaire',
 '50% des poissons vivraient dans 17% de la surface des océans ',
 'un tissu épithélial',
 'sa présence, de sa beauté, de sa jovialité ou encore pour ses talents',
 'miaou',
 'campagne',
 'forme solide ou liquide',
 'Le chat',
 'entre l’ œsophage et le duodénum',
 'des sons',
 '',
 'le contact',
 'un handicap auditif',
 'la phase du sommeil au cours de laquelle les  rêves  dont on se souvient se produisent',
 'du latin species , «type» ou «apparence»',
 'de taille moyenne ou petite',
 'des noms scientifiques différents pour désigner un même taxon',
 'Antiquité tardive',
 'sous la surface intérieure du nez',
 'la pureté de la race',
 'rayonnements lumineux',
 '',
 'des glandes exocrines',
 'éthologique',
 'en 1894',
 '',
 'la transpiration',
 'en crocs',
 'la mélanine et la phéomélanine',
 'les  fossiles , les  organismes   asexués  ou pour des espèces rares ou difficiles 